# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [3]:
! pip install -U langchain

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


## Chain Class 基类

类继承关系：

```
Chain --> <name>Chain  # Examples: LLMChain, MapReduceChain, RouterChain
```

**代码实现：https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/base.py**

```python
# 定义一个名为Chain的基础类
class Chain(Serializable, Runnable[Dict[str, Any], Dict[str, Any]], ABC):
    """为创建结构化的组件调用序列的抽象基类。
    
    链应该用来编码对组件的一系列调用，如模型、文档检索器、其他链等，并为此序列提供一个简单的接口。
    
    Chain接口使创建应用程序变得容易，这些应用程序是：
    - 有状态的：给任何Chain添加Memory可以使它具有状态，
    - 可观察的：向Chain传递Callbacks来执行额外的功能，如记录，这在主要的组件调用序列之外，
    - 可组合的：Chain API足够灵活，可以轻松地将Chains与其他组件结合起来，包括其他Chains。
    
    链公开的主要方法是：
    - `__call__`：链是可以调用的。`__call__`方法是执行Chain的主要方式。它将输入作为一个字典接收，并返回一个字典输出。
    - `run`：一个方便的方法，它以args/kwargs的形式接收输入，并将输出作为字符串或对象返回。这种方法只能用于一部分链，不能像`__call__`那样返回丰富的输出。
    """

    # 调用链
    def invoke(
        self, input: Dict[str, Any], config: Optional[runnableConfig] = None
    ) -> Dict[str, Any]:
        """传统调用方法。"""
        return self(input, **(config or {}))

    # 链的记忆，保存状态和变量
    memory: Optional[BaseMemory] = None
    """可选的内存对象，默认为None。
    内存是一个在每个链的开始和结束时被调用的类。在开始时，内存加载变量并在链中传递它们。在结束时，它保存任何返回的变量。
    有许多不同类型的内存，请查看内存文档以获取完整的目录。"""

    # 回调，可能用于链的某些操作或事件。
    callbacks: Callbacks = Field(default=None, exclude=True)
    """可选的回调处理程序列表（或回调管理器）。默认为None。
    在对链的调用的生命周期中，从on_chain_start开始，到on_chain_end或on_chain_error结束，都会调用回调处理程序。
    每个自定义链可以选择调用额外的回调方法，详细信息请参见Callback文档。"""

    # 是否详细输出模式
    verbose: bool = Field(default_factory=_get_verbosity)
    """是否以详细模式运行。在详细模式下，一些中间日志将打印到控制台。默认值为`langchain.verbose`。"""

    # 与链关联的标签
    tags: Optional[List[str]] = None
    """与链关联的可选标签列表，默认为None。
    这些标签将与对这个链的每次调用关联起来，并作为参数传递给在`callbacks`中定义的处理程序。
    你可以使用这些来例如识别链的特定实例与其用例。"""

    # 与链关联的元数据
    metadata: Optional[Dict[str, Any]] = None
    """与链关联的可选元数据，默认为None。
    这些元数据将与对这个链的每次调用关联起来，并作为参数传递给在`callbacks`中定义的处理程序。
    你可以使用这些来例如识别链的特定实例与其用例。"""
```

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

In [5]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.9, max_tokens=500,api_key='sk-1eaLNHqUQ7LXBLS38c47Bc660c63451897EeAc31Db192244',
            # 代理地址，填写商家中转站或自建OpenAI代理
            base_url='https://api.xiaoai.plus/v1')

In [6]:
prompt = PromptTemplate(
    input_variables=["product"],
    template="给制造{product}的有限公司取10个好名字，并给出完整的公司名称",
)

In [7]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)
print(chain.invoke({
    'product': "性能卓越的GPU"
    }))

D:\Program_Files\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


{'product': '性能卓越的GPU', 'text': '\n\n1. 创能图形科技有限公司\n2. 异能绘图有限公司\n3. 极光视界技术有限公司\n4. 光速绘图有限公司\n5. 天元设计图形有限公司\n6. 神威图形科技有限公司\n7. 竞技绘图有限公司\n8. 元素视觉科技有限公司\n9. 未来图像有限公司\n10. 梦幻绘图科技有限公司'}


In [5]:
chain.verbose =True

In [6]:
chain.verbose

True

In [7]:
print(chain.invoke({
    'product': "性能卓越的GPU"
    }))



> Entering new LLMChain chain...
Prompt after formatting:
给制造性能卓越的GPU的有限公司取10个好名字，并给出完整的公司名称

> Finished chain.
{'product': '性能卓越的GPU', 'text': '\n\n\n1. 锐芯科技有限公司（Sharpcore Technology Co., Ltd.）\n2. 超晶科技有限公司（Supercrystal Technology Co., Ltd.）\n3. 星云科技有限公司（Starcloud Technology Co., Ltd.）\n4. 弦火科技有限公司（Chordfire Technology Co., Ltd.）\n5. 极光声浪有限公司（Aurora Soundwave Co., Ltd.）\n6. 炫影视界有限公司（Luminous Vision Co., Ltd.）\n7. 梦幻芯片有限公司（Dreamchip Co., Ltd.）\n8. 科创纵横有限公司（Innovate Horizon Co., Ltd.）\n9. 虚拟巅峰有限公司（Virtual Summit Co., Ltd.）\n10. 神威科技有限公司（Divineforce Technology Co., Ltd.）'}


## Sequential Chain

串联式调用语言模型（将一个调用的输出作为另一个调用的输入）。

顺序链（Sequential Chain ）允许用户连接多个链并将它们组合成执行特定场景的流水线（Pipeline）。有两种类型的顺序链：

- SimpleSequentialChain：最简单形式的顺序链，每个步骤都具有单一输入/输出，并且一个步骤的输出是下一个步骤的输入。
- SequentialChain：更通用形式的顺序链，允许多个输入/输出。

### 使用 SimpleSequentialChain 实现戏剧摘要和评论（单输入/单输出）

![](../images/simple_sequential_chain_0.png)

In [8]:
# 这是一个 LLMChain，用于根据剧目的标题撰写简介。

llm = OpenAI(temperature=0.7, max_tokens=1000)

template = """你是一位剧作家。根据戏剧的标题，你的任务是为该标题写一个简介。

标题：{title}
剧作家：以下是对上述戏剧的简介："""

prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

In [9]:
# 这是一个LLMChain，用于根据剧情简介撰写一篇戏剧评论。
# llm = OpenAI(temperature=0.7, max_tokens=1000)
template = """你是《纽约时报》的戏剧评论家。根据剧情简介，你的工作是为该剧撰写一篇评论。

剧情简介：
{synopsis}

以下是来自《纽约时报》戏剧评论家对上述剧目的评论："""

prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template)

![](../images/simple_sequential_chain_1.png)

In [8]:
# 这是一个SimpleSequentialChain，按顺序运行这两个链
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[synopsis_chain, review_chain], verbose=True)

NameError: name 'synopsis_chain' is not defined

In [11]:
review = overall_chain.invoke("三体人不是无法战胜的")



> Entering new SimpleSequentialChain chain...


《三体人不是无法战胜的》是一部关于宇宙中最强大的种族——三体人的故事。在这个宇宙中，三体人被认为是无法战胜的，因为他们拥有超越常人的科技和力量。然而，一位普通人类的出现改变了一切。他发现了三体人的弱点，并决心与他们抗争，为人类的自由而战。在与三体人的殊死搏斗中，他发现三体人并非不可战胜，而是因为他们的内心缺乏爱和同情。最终，普通人类的勇气和坚持，打破了三体人的统治，带来了和平与和谐的新时代。这部戏剧探讨了人类与外星种族的关系，以及爱和同情的力量对战争的影响。它将带领观众进入一个惊险刺激的宇宙冒险，同时也给人们带来思考和启发。


《三体人不是无法战胜的》是一部充满惊险和启发的戏剧作品。它通过一个普通人类与宇宙中最强大的种族——三体人的对抗，探讨了人类与外星种族的关系，以及爱和同情的力量对战争的影响。

这部作品精彩地展现了三体人的科技与力量，以及他们对人类的统治。观众们将被带入一个充满刺激的宇宙冒险，与主角一起感受战斗的紧张与惊险。同时，剧中也深刻地探讨了三体人的弱点，以及人类如何通过勇气和坚持最终战胜他们。

最令人感动的是，剧中展现了爱和同情的力量。主角发现，三体人之所以无法战胜，并不是因为他们的科技和力量，而是因为他们内心缺乏爱和同情。这也引发了人们对战争的思考，让观众们深刻地反思爱和同情对于和平与和谐的重要性。

整部戏剧的制作也十分精彩，舞台设计、服装和道具都充分展现了宇宙的奇幻和未来的科技感。演员们的表演也令人印象深刻，他们生动地诠释了各个角色的复杂心理和情感。

总的来说，《三体人不是无法战胜的》是一部令人难忘的戏剧作品。它不仅带给观众惊险的冒险和刺激的战斗，更重要的是，它让我们思考人类与外星种族的关系，以及爱和同情的力量对于和平与和谐的重要性。这部作品将在观众心中留下深刻的印象，值得一看。

> Finished chain.


In [12]:
review = overall_chain.invoke("星球大战第九季")



> Entering new SimpleSequentialChain chain...

《星球大战第九季》是一部充满惊险刺激的科幻戏剧，它延续了《星球大战》系列的传奇故事。在这一季中，我们将见证新的冒险和挑战，因为银河系再次陷入战争的深渊。反抗军和第一秩序之间的战争仍在继续，但这一次，他们将面临着更加强大的敌人。新的角色和老朋友将再次出现，与我们的英雄们一起并肩作战。随着战争的进展，我们也将见证力量和黑暗的永恒对抗，以及对自由和正义的不懈追求。在这场史诗般的战役中，我们将看到勇气、牺牲和爱的力量，同时也会思考我们对未来的选择和命运。《星球大战第九季》将带给观众无与伦比的视觉盛宴和感人心弦的故事，让我们一起来见证这一壮丽的星际冒险！


《星球大战第九季》是一部不容错过的史诗般的科幻戏剧。该剧延续了《星球大战》系列的传奇故事，将观众带入一个充满惊险和刺激的银河系。在这一季中，我们将再次与我们熟悉的英雄们一起并肩作战，同时也会见证新的挑战和冒险。

该剧不仅仅是一场视觉盛宴，更是一部让人思考的故事。通过力量和黑暗的永恒对抗，以及对自由和正义的追求，观众将被带入一个关于命运和选择的深刻思考。在这场史诗般的战役中，我们将见证勇气、牺牲和爱的力量，这些都是《星球大战》系列一直探讨的重要主题。

该剧不仅有着令人惊叹的特效和精彩的动作场面，更重要的是它成功地将这些元素与深刻的人物关系和情感表达相结合。观众将会被带入一个充满希望和感动的星际冒险，与角色们一起经历他们的成长和转变。

总的来说，《星球大战第九季》是一部充满情感和故事的出色作品，它将让观众沉浸在一个充满惊喜和挑战的星际世界。无论您是《星球大战》系列的忠实粉丝，还是对科幻题材感兴趣的观众，这部戏剧都会给您带来一场令人难忘的体验。不容错过！

> Finished chain.


### 使用 SequentialChain 实现戏剧摘要和评论（多输入/多输出）

![](../images/sequential_chain_0.png)

In [9]:
# # 这是一个 LLMChain，根据剧名和设定的时代来撰写剧情简介。
llm = OpenAI(temperature=.7, max_tokens=1000,api_key='sk-1eaLNHqUQ7LXBLS38c47Bc660c63451897EeAc31Db192244',
            # 代理地址，填写商家中转站或自建OpenAI代理
            base_url='https://api.xiaoai.plus/v1')
template = """你是一位剧作家。根据戏剧的标题和设定的时代，你的任务是为该标题写一个简介。

标题：{title}
时代：{era}
剧作家：以下是对上述戏剧的简介："""

prompt_template = PromptTemplate(input_variables=["title", "era"], template=template)
# output_key
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="synopsis", verbose=True)

In [10]:
# 这是一个LLMChain，用于根据剧情简介撰写一篇戏剧评论。

template = """你是《纽约时报》的戏剧评论家。根据该剧的剧情简介，你需要撰写一篇关于该剧的评论。

剧情简介：
{synopsis}

来自《纽约时报》戏剧评论家对上述剧目的评价："""

prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="review", verbose=True)

In [11]:
from langchain.chains import SequentialChain

m_overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True)

In [12]:
m_overall_chain.invoke({"title":"三体人不是无法战胜的", "era": "二十一世纪的新中国"})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
你是一位剧作家。根据戏剧的标题和设定的时代，你的任务是为该标题写一个简介。

标题：三体人不是无法战胜的
时代：二十一世纪的新中国
剧作家：以下是对上述戏剧的简介：

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
你是《纽约时报》的戏剧评论家。根据该剧的剧情简介，你需要撰写一篇关于该剧的评论。

剧情简介：


在二十一世纪的新中国，人类社会已经迈入了一个全新的时代。科技的发展让人类有了更多的可能性，但也带来了新的挑战。在这个充满变数的时代，一场前所未有的战争即将爆发。

这场战争的参与者并非是人类，而是来自遥远星球的三体人。他们拥有强大的科技实力，凭借着超越人类的智慧和力量，他们看似不可战胜。

然而，在这个时代的中国，人们并没有被吓倒。他们不仅拥有着强大的科技实力，更有着坚定的信念和无比的勇气。他们决心不让三体人的侵略毁灭自己的家园，而是要与他们展开一场生死决战。

在这场战争中，人类将面临着巨大的挑战和牺牲，但也将展现出令人敬佩的勇气和坚强的意志。他们不会被三体人所打败，因为他们相信，只要有信念和团结，就没有什么是无法战胜的。

《三体人不是无法战胜的》是一部关于人类意志和勇气的战争剧，它将带领观众走进一个未来世界，见证人类与外星文明的生死搏斗，也将带给观众深刻的思考和感动。让我们一起期待这场战争的结局，以及人类的未来。

来自《纽约时报》戏剧评论家对上述剧目的评价：

> Finished chain.

> Finished chain.


{'title': '三体人不是无法战胜的',
 'era': '二十一世纪的新中国',
 'synopsis': '\n\n在二十一世纪的新中国，人类社会已经迈入了一个全新的时代。科技的发展让人类有了更多的可能性，但也带来了新的挑战。在这个充满变数的时代，一场前所未有的战争即将爆发。\n\n这场战争的参与者并非是人类，而是来自遥远星球的三体人。他们拥有强大的科技实力，凭借着超越人类的智慧和力量，他们看似不可战胜。\n\n然而，在这个时代的中国，人们并没有被吓倒。他们不仅拥有着强大的科技实力，更有着坚定的信念和无比的勇气。他们决心不让三体人的侵略毁灭自己的家园，而是要与他们展开一场生死决战。\n\n在这场战争中，人类将面临着巨大的挑战和牺牲，但也将展现出令人敬佩的勇气和坚强的意志。他们不会被三体人所打败，因为他们相信，只要有信念和团结，就没有什么是无法战胜的。\n\n《三体人不是无法战胜的》是一部关于人类意志和勇气的战争剧，它将带领观众走进一个未来世界，见证人类与外星文明的生死搏斗，也将带给观众深刻的思考和感动。让我们一起期待这场战争的结局，以及人类的未来。',
 'review': '\n\n《三体人不是无法战胜的》是一部令人叹为观止的战争剧。它不仅展现了科技发展带来的可能性和挑战，更深刻地揭示了人类的意志和勇气。在这个未来世界，人类将面临来自遥远星球的三体人的侵略，但他们并没有被吓倒，而是展现出了令人敬佩的坚强意志。这场战争不仅是一场生死搏斗，更是一次对人类信念和团结的考验。\n\n剧中的人物形象栩栩如生，让观众能够深入地感受到他们的内心挣扎和决心。他们的勇气和无畏精神令人动容，让人不禁为他们的牺牲和奋斗感到敬佩。剧中的特效也非常出色，将观众带入一个充满奇幻和惊险的未来世界。\n\n除了战争的场面，该剧也深刻地探讨了人类与外星文明的关系。三体人拥有超越人类的科技实力，但也让人们思考，科技是否真的能够解决一切问题？人类与外星文明的交流和冲突，也让人们反思自身的文明和价值观。\n\n总的来说，《三体人不是无法战胜的》是一部充满哲理的战争剧，它不仅令人兴奋，更让人深思。它将观众带入一个未来世界，让我们见证人类的意志和勇气，也让我们思考自己的未来。这部戏剧必将成为经典，值得每个人去观看。'}

### Homework

#### 使用 OutputParser 优化 overall_chain 输出格式，区分 synopsis_chain 和 review_chain 的结果